In [5]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam, SGD, RMSprop
from tensorflow.keras import regularizers
from tensorflow.keras.datasets import mnist

import numpy as np
import matplotlib.pyplot as plt

import mlflow
import dagshub
import time

print(f"TensorFlow: {tf.__version__}")
print(f"MLflow: {mlflow.__version__}")

TensorFlow: 2.20.0
MLflow: 3.6.0


In [6]:
# Verificar disponibilidad de GPU y listar dispositivos físicos
gpus = tf.config.list_physical_devices('GPU')
print(f"GPUs disponibles: {len(gpus)}")

if gpus:
    for i, gpu in enumerate(gpus):
        print(f"  GPU {i}: {gpu}")
    
    # Configurar crecimiento de memoria (evita que TensorFlow reserve toda la VRAM)
    tf.config.experimental.set_memory_growth(gpu, True)
    
    # Información adicional sobre la GPU
    print("\nDetalles de GPU:")
    from tensorflow.python.client import device_lib
    local_devices = device_lib.list_local_devices()
    for device in local_devices:
        if device.device_type == 'GPU':
            print(f"  Nombre: {device.physical_device_desc}")
else:
    print("No se detectó ninguna GPU. El entrenamiento usará CPU.")

GPUs disponibles: 1
  GPU 0: PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')

Detalles de GPU:
  Nombre: device: 0, name: NVIDIA GeForce GTX 1050 Ti, pci bus id: 0000:01:00.0, compute capability: 6.1


I0000 00:00:1763952718.878973   27461 gpu_device.cc:2020] Created device /device:GPU:0 with 2857 MB memory:  -> device: 0, name: NVIDIA GeForce GTX 1050 Ti, pci bus id: 0000:01:00.0, compute capability: 6.1


In [9]:
# Cargamos lso datos

(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Aplanar y normalizar
x_train = x_train.reshape(60000, 784).astype('float32') / 255.0
x_test = x_test.reshape(10000, 784).astype('float32') / 255.0

# Onehot encoding
num_classes = 10
y_train_cat = keras.utils.to_categorical(y_train, num_classes)
y_test_cat = keras.utils.to_categorical(y_test, num_classes)